In [18]:
from variational_autoencoder import VAE
import torch.nn.functional as F
import torch
from tqdm import tqdm
import os

In [ ]:
def get_data(path):
    

In [13]:
def reconstruction_loss(x, x_hat):
    return F.mse_loss(x_hat, x, reduction='sum')

def kl_divergence_loss(mu, stddev, beta=0.5):
    return beta * torch.sum(torch.exp(stddev) + mu**2 - 1.0 - stddev)

def loss_function(x, x_hat, mu, stddev, beta=0.5):
    return reconstruction_loss(x, x_hat) + kl_divergence_loss(mu, stddev, beta)

In [14]:
def train(model, device, optimizer, train_dataloader, epochs, loss_output_interval):
    model.to(device)
    model.train()
    
    for step in tqdm(range(epochs), desc="Epoch"):
        running_loss = 0
        total_running_loss = 0
        
        for i, (X, y) in enumerate(tqdm(train_dataloader, desc="Batch", leave=False)):
            X = X.to(device)
            y = y.to(device)
            X_hat, mu, stddev = model(X)
            loss = loss_function(X, X_hat, mu, stddev)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            total_running_loss += loss.item()
        
        if (step+1) % loss_output_interval == 0:
            print(f' Epoch {step+1} Average Batch Loss: {total_running_loss/len(train_dataloader)}')
            
    model.eval()

In [37]:
list(x[0] for x in os.walk('../data/'))[4:]

['../data/simpsons_dataset/kent_brockman',
 '../data/simpsons_dataset/principal_skinner',
 '../data/simpsons_dataset/agnes_skinner',
 '../data/simpsons_dataset/lenny_leonard',
 '../data/simpsons_dataset/krusty_the_clown',
 '../data/simpsons_dataset/sideshow_mel',
 '../data/simpsons_dataset/patty_bouvier',
 '../data/simpsons_dataset/nelson_muntz',
 '../data/simpsons_dataset/abraham_grampa_simpson',
 '../data/simpsons_dataset/miss_hoover',
 '../data/simpsons_dataset/gil',
 '../data/simpsons_dataset/bart_simpson',
 '../data/simpsons_dataset/professor_john_frink',
 '../data/simpsons_dataset/carl_carlson',
 '../data/simpsons_dataset/moe_szyslak',
 '../data/simpsons_dataset/ned_flanders',
 '../data/simpsons_dataset/sideshow_bob',
 '../data/simpsons_dataset/groundskeeper_willie',
 '../data/simpsons_dataset/apu_nahasapeemapetilon',
 '../data/simpsons_dataset/rainier_wolfcastle',
 '../data/simpsons_dataset/cletus_spuckler',
 '../data/simpsons_dataset/lisa_simpson',
 '../data/simpsons_dataset/mi

(tensor([[-0.0723,  0.0869, -0.3901, -0.1682, -0.0650, -0.2496, -0.1893,  0.1371,
          -0.1893,  0.0838, -0.1771, -0.1335, -0.3467, -0.2172, -0.2079,  0.0240]],
        grad_fn=<AddmmBackward0>),
 tensor([[-0.2126, -0.3072,  0.1850, -0.3313]], grad_fn=<AddmmBackward0>),
 tensor([[-0.2632, -0.1324, -0.1859, -0.1747]], grad_fn=<AddmmBackward0>))